### 20.1.1 LineItem类第3版：一个简单的描述符

In [1]:
class Quantity:
    def __init__(self, storage_name):
        self.storage_name = storage_name
    def __set__(self, instance, value):
        if value > 0:
            instance.__dict__[self.storage_name] = value
        else:
            raise ValueError('value must be > 0')

class LineItem:
    weight = Quantity('weight')
    price = Quantity('price')
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
    def subtotal(self):
        return self.weight * self.price

In [2]:
truffle = LineItem('white truffle', 100, 0)

ValueError: value must be > 0

### 20.1.2 LineItem类第4版：自动获取储存属性的名称

In [3]:
class Quantity:
    __counter = 0
    def __init__(self):
        cls = self.__class__
        prefix = cls.__name__
        index = cls.__counter
        self.storage_name = '_{}#{}'.format(prefix, index)
        cls.__counter+= 1
    def __get__(self, instance, owner):
        return getattr(instance, self.storage_name)
    def __set__(self, instance, value):
       if value > 0:
           setattr(instance, self.storage_name, value)
       else:
           raise ValueError('value must be > 0')
class LineItem:
    weight = Quantity()
    price = Quantity()
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
    def subtotal(self):
        return self.weight * self.price

In [4]:
coconuts = LineItem('Brazilian coconut', 20, 17.95)

In [5]:
coconuts.weight, coconuts.price

(20, 17.95)

In [6]:
getattr(coconuts, '_Quantity#0'), getattr(coconuts, '_Quantity#1')

(20, 17.95)

In [9]:
# bulkfood_v4b.py（只列出部分代码）：通过托管类调用时，__get__方法返回描述符的引用

class Quantity:
    __counter = 0
    def __init__(self):
        cls = self.__class__
        prefix = cls.__name__
        index = cls.__counter
        self.storage_name = '_{}#{}'.format(prefix, index)
        cls.__counter+= 1
    def __get__(self, instance, owner):
        if instance is None:
            return self
        else:
            return getattr(instance, self.storage_name)
    def __set__(self, instance, value):
        if value > 0:
            setattr(instance, self.storage_name, value)
        else:
            raise ValueError('value must be > 0')

In [ ]:
LineItem.price

In [11]:
br_nuts = LineItem('Brazil nuts', 10, 34.95)
br_nuts.price

34.95

### 20.1.3 LineItem类第5版：一种新型描述符

In [12]:
import abc
class AutoStorage:
    __counter = 0
    def __init__(self):
        cls = self.__class__
        prefix = cls.__name__
        index = cls.__counter
        self.storage_name = '_{}#{}'.format(prefix, index)
        cls.__counter+= 1
    def __get__(self, instance, owner):
        if instance is None:
            return self
        else:
            return getattr(instance, self.storage_name)
    def __set__(self, instance, value):
        setattr(instance, self.storage_name, value)
class Validated(abc.ABC, AutoStorage):
    def __set__(self, instance, value):
        value = self.validate(instance, value)
        super().__set__(instance, value)
    @abc.abstractmethod
    def validate(self, instance, value):
        """return validated value or raise ValueError"""
class Quantity(Validated):
    """a number greater than zero"""
    def validate(self, instance, value):
        if value <= 0:
            raise ValueError('value must be > 0')
        return value
class NonBlank(Validated):
    """a string with at least one non-space character"""
    def validate(self, instance, value):
        value = value.strip()
        if len(value) == 0:
            raise ValueError('value cannot be empty or blank')
        return value